In [ ]:
import numpy as np
import os
import cv2
%matplotlib inline
from skimage.io import imread, imshow
from skimage.transform import resize
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import time
import tensorflow as tf

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import os
from patchify import patchify, unpatchify
import random
import numpy as np

import glob
import os
from pathlib import Path
import math
from skimage.io import imread, imshow
from skimage.transform import resize


seed = 42
np.random.seed = seed
from keras.utils.np_utils import normalize
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras import backend as K
from numpy import asarray

from keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:

      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    

Delete Old folder first

In [ ]:
def get_valid_sz(shape):
    rat_x = math.ceil(shape[0]/step_size)
    x_size =int(step_size*rat_x)
    x_size = x_size  + patch_sz - step_size
    
    rat_y = math.ceil(shape[1]/step_size)
    y_size =int(step_size * rat_y)
    y_size = y_size  + patch_sz - step_size
    
    return int(x_size), int(y_size)

In [ ]:
def pad_image(image, patch_size):
    x_size, y_size = get_valid_sz(image.shape)
    height, width = image.shape
    padded_image = np.pad(image, ((0, x_size - height ), (0, y_size - width)), mode='constant')
    return padded_image

In [ ]:
def Patch_Generator(data_pth, lab_path, save_data_pth, save_masks_pth, include_large_img = False, with_labels = True):
    if not os.path.exists(save_data_pth):
        os.makedirs(save_data_pth)
    if with_labels:
        if not os.path.exists(save_masks_pth):
            os.makedirs(save_masks_pth)
    c = 0
 
    for img_path in sorted(glob.glob(os.path.join(data_pth))):
        try:
           
            image = cv2.imread(img_path, 0)
            
            aspect_ratio = image.shape[1] / image.shape[0]
    
            new_size = 2400
            if aspect_ratio > 1:
                new_width = new_size
                new_height = int(new_width / aspect_ratio)
            else:
                new_height = new_size
                new_width = int(new_height * aspect_ratio)
            image = cv2.resize(image, (new_width, new_height))
            
            image = pad_image(image, IMG_HEIGHT)

            
            patches = patchify(image, (IMG_HEIGHT,IMG_WIDTH), step=step_size)
           
            patches_img = patches.reshape(patches.shape[0]*patches.shape[1],IMG_HEIGHT, IMG_WIDTH )
            
            if with_labels:
                fn = img_path.split("\\")[-1]
                mask_path = os.path.join(lab_path, 'M'+fn)
                mask = cv2.imread(mask_path, 0)
                
                mask = cv2.resize(mask, (new_width, new_height))

                mask = pad_image(mask, IMG_HEIGHT)

                patches_mask = patchify(mask, (IMG_HEIGHT, IMG_WIDTH), step=step_size) 

                patches_mask =  patches_mask.reshape(patches_mask.shape[0]*patches_mask.shape[1],IMG_HEIGHT, IMG_WIDTH )
            
            if include_large_img:
                cv2.imwrite(save_data_pth+str(c)+'_org.png', image)
                if with_labels:
                    cv2.imwrite(save_masks_pth+str(c)+'_org.png', mask)
            
            for i in range(len(patches_img)):
           
                img = patches_img[i]
                if with_labels:
                    mask = patches_mask[i]
                 
                    if (np.sum(mask) !=0):
                        image_ = Image.fromarray(img, mode='L')
                        # Save the grayscale image as a PNG file
                        image_.save(save_data_pth+str(c)+'_.png')

                        mask_ = Image.fromarray(mask, mode='L')
                        mask_.save(save_masks_pth+str(c)+'_.png')
                        c=c+1
                else:
                    image_ = Image.fromarray(img, mode='L')
                        # Save the grayscale image as a PNG file
                    image_.save(save_data_pth+str(c)+'_.png')
                    c=c+1
        except:
            pass
    

## Sample Usage - May take time

In [ ]:
data_path = r'< INTPUT_IMAGES_PATH >'   
lab_path = r"< INPUT_LABELS >"
save_data_pth = "< PATH_TO_DIR >" 
save_masks_pth = "< PATH_TO_DIR >"

# Create empty arrays
IMG_HEIGHT = 512
IMG_WIDTH = 512
IMG_CHANNELS = 1
step_size = 386
patch_sz = 512
include_full_size_img = False

per_image_norm = False

Patch_Generator(data_path, lab_path, save_data_pth, save_masks_pth, include_large_img= False)